# Application of Genetic Algorithm and Neural Networks in Solving the XOR Problem

This notebook explores the application of genetic algorithm and neural networks in solving the XOR problem.

The XOR gate has the following logic table

Input: (0,0) -> Output: 0

Input: (1,0) -> Output: 1

Input: (0,1) -> Output: 1

Input: (1,1) -> Output: 0

The topology of the neural network is described below

input layer (2 nodes) -> hidden layer 1 (3 nodes) -> hidden layer 2 (2 nodes) -> output layer (1 node)

In [1]:
# import libraries
import math
import copy
import random
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from helper_module import split_train_test_sets, split_train_set
from typing import Tuple, Iterable, NewType

In [2]:
# instantiate the train set
x = ((0,0),(1,0),(0,1),(1,1))
y = (0,1,1,0)

In [3]:
# define the sigmoid activation function
def sigmoid(z:float) -> float:
    return 1/(1 + np.exp(-z))

In [4]:
# define the neural network classs
class NN():
    def __init__(self):
        self.W_layer1: np.ndarray
        self.W_layer2: np.ndarray
        self.W_layer3: np.ndarray
        self.b_layer1: np.ndarray
        self.b_layer2: np.ndarray
        self.b_layer3: np.ndarray
        self.fitness: float

    def __str__(self):
        text = str()
        text = text + f'{self.W_layer1}'
        text = text + f'{self.W_layer2}'
        text = text + f'{self.W_layer3}'
        text = text + f'{self.b_layer1}'
        text = text + f'{self.b_layer2}'
        text = text + f'{self.b_layer3}'
        return text

In [5]:
from functools import reduce

# define a function that normalizes a vector
def normalize_vector(x:np.ndarray) -> np.ndarray:
    vector_magnitude_list = list()
    vector_magnitude_list = [math.sqrt(i**2) for i in x]
    vector_len = reduce(lambda x, y: x+y, vector_magnitude_list)
    normalized_vector = np.array([(i/vector_len) for i in x])

    return normalized_vector

In [6]:
def evaluate_fitness(x:Tuple[Tuple[int]], y: Tuple[int], nn:NN) -> float:
    # initialize the fitness score to zero
    fitness = 0

    # loop through all the training data
    # and apply forward propagation of NN
    for (x1, x2), y1 in zip(x, y):
        input_layer = np.array([[x1],[x2]])
        Z1 = np.add(np.matmul(nn.W_layer1, input_layer), nn.b_layer1)
        A1 = np.array([sigmoid(i) for i in Z1])
        A1 = normalize_vector(A1)
        Z2 = np.add(np.matmul(nn.W_layer2, A1), nn.b_layer2)
        A2 = np.array([sigmoid(i) for i in Z2])
        A2 = normalize_vector(A2)
        Z3 = np.add(np.matmul(nn.W_layer3, A2), nn.b_layer3)
        A3 = sigmoid(Z3)
        
        # perform binary classification
        if A3 > 0.5:
            y_pred = 1
        else:
            y_pred = 0

        # compute fitness
        fitness += math.sqrt((y1 - y_pred)**2)

    return fitness

In [7]:
def init_NN() -> NN:
    # instantiate NN object
    nn = NN()

    # instantiate the weight matrixes
    nn.W_layer1 = np.random.uniform(-10, 10, size=(3,2))
    nn.W_layer2 = np.random.uniform(-10, 10, size=(2,3))
    nn.W_layer3 = np.random.uniform(-10, 10, size=(1,2))
    
    # instantiate the bias vectors
    nn.b_layer1 = np.random.uniform(-10, 10, size=(3,1))
    nn.b_layer2 = np.random.uniform(-10, 10, size=(2,1))
    nn.b_layer3 = np.random.uniform(-10, 10, size=(1,1))

    return nn

In [8]:
def flatten_NN(nn:NN) -> np.ndarray:
    flat_nn = np.array([])
    flat_nn = np.append(flat_nn, nn.W_layer1.flatten())
    flat_nn = np.append(flat_nn, nn.W_layer2.flatten())
    flat_nn = np.append(flat_nn, nn.W_layer3.flatten())
    flat_nn = np.append(flat_nn, nn.b_layer1.flatten())
    flat_nn = np.append(flat_nn, nn.b_layer2.flatten())
    flat_nn = np.append(flat_nn, nn.b_layer3.flatten())

    return flat_nn

In [9]:
def create_NN(flat_nn) -> NN:
    new_nn = NN()
    W_layer_1 = np.empty((3,2))
    W_layer_2 = np.empty((2,3))
    W_layer_3 = np.empty((1,2))

    b_layer_1 = np.empty((3,1))
    b_layer_2 = np.empty((2,1))
    b_layer_3 = np.empty((1,1))

    W_layer_1[0][0] = flat_nn[0] 
    W_layer_1[0][1] = flat_nn[1] 
    W_layer_1[1][0] = flat_nn[2] 
    W_layer_1[1][1] = flat_nn[3] 
    W_layer_1[2][0] = flat_nn[4] 
    W_layer_1[2][1] = flat_nn[5] 

    W_layer_2[0][0] = flat_nn[6] 
    W_layer_2[0][1] = flat_nn[7] 
    W_layer_2[0][2] = flat_nn[8]
    W_layer_2[1][0] = flat_nn[9] 
    W_layer_2[1][1] = flat_nn[10]
    W_layer_2[1][2] = flat_nn[11]

    W_layer_3[0][0] = flat_nn[12] 
    W_layer_3[0][1] = flat_nn[13] 

    b_layer_1[0] = flat_nn[14] 
    b_layer_1[1] = flat_nn[15] 
    b_layer_1[2] = flat_nn[16] 

    b_layer_2[0] = flat_nn[17] 
    b_layer_2[1] = flat_nn[18] 

    b_layer_3[0] = flat_nn[19] 
    
    new_nn = NN()
    new_nn.W_layer1 = W_layer_1
    new_nn.W_layer2 = W_layer_2
    new_nn.W_layer3 = W_layer_3
    new_nn.b_layer1 = b_layer_1
    new_nn.b_layer2 = b_layer_2
    new_nn.b_layer3 = b_layer_3

    return new_nn

In [10]:
# perform uniform crossover
def crossover(flat_nn1, flat_nn2):
    # instantiate the offsprings as lists
    offspring1 = list()
    offspring2 = list()

    # loop through the nodes of the boths NNs
    # and perform uniform crossover
    for nn1_node, nn2_node in zip(flat_nn1, flat_nn2):
        if random.uniform(0,1) >=0.5:
            offspring1.append(nn1_node)
            offspring2.append(nn2_node)
        else:
            offspring1.append(nn2_node)
            offspring2.append(nn1_node)
    
    # instatiate the offsprings as genomes
    offspring1 = np.array(offspring1)
    offspring2 = np.array(offspring2)

    return offspring1, offspring2

In [11]:
# perform mutation
def mutate(flat_nn, mutation_rate: float = 0.1):

    new_flat_nn = copy.deepcopy(flat_nn)
    for idx, _ in enumerate(flat_nn):
        if np.random.uniform(0,1) > (1 - mutation_rate):
            new_flat_nn[idx] = np.random.uniform(-10, 10)
        else:
            new_flat_nn[idx] = copy.deepcopy(flat_nn[idx])
    
    return new_flat_nn

In [12]:
def reproduce(nn1, nn2):
    flat_nn1 = flatten_NN(nn1)
    flat_nn2 = flatten_NN(nn2)
    offspring1, offspring2 = crossover(flat_nn1, flat_nn2)
    offspring1 = mutate(offspring1)
    offspring2 = mutate(offspring2)
    offspring1 = create_NN(offspring1)
    offspring2 = create_NN(offspring2)

    return offspring1, offspring2

In [13]:
def init_population(num_individuals:int) -> Iterable[NN]:
    population = list()
    for i in range(num_individuals):
        population.append(init_NN())

    return population

In [14]:
def keep_elites(percentage_elites:float, population: Iterable[NN]):    
    sorted_population = sorted(population, key=lambda x: x.fitness, reverse=False)
    elite_population = list()
    num_elites = math.floor(percentage_elites * len(population))
    sorted_population = sorted(population, key=lambda x: x.fitness, reverse=False)
    elite_population = sorted_population[:num_elites:]

    return elite_population

In [15]:
# implement tournament selection
def selection(population: Iterable[NN]):
    random.shuffle(population)
    selected_population = list()
    left_bracket = population[:math.floor(len(population)/2):]
    right_bracket = population[math.floor(len(population)/2)::]

    for left_nn, right_nn in zip(left_bracket, right_bracket):
        if left_nn.fitness < right_nn.fitness:
            selected_population.append(left_nn)
        else:
            selected_population.append(right_nn)

    return selected_population

In [16]:
def reproduce_population(population: Iterable[NN]) -> Iterable[NN]:
    new_population = list()
    offspring_population = list()
    left_bracket = population[:math.floor(len(population)/2):]
    right_bracket = population[math.floor(len(population)/2)::]

    for left_nn, right_nn in zip(left_bracket, right_bracket):
        offsprin1, offspring2 = reproduce(left_nn, right_nn)
        offspring_population.append(offsprin1)
        offspring_population.append(offspring2)

    new_population = population + offspring_population

    return new_population

In [17]:
def reduce_population(population:Iterable[NN], remove_percentage:float):
    new_population = population[:-math.floor(len(population)*remove_percentage):]

    return new_population

In [18]:
def run_evolution(population: Iterable[NN], num_generations:int):
    for _ in range(num_generations):
        # compute the fitness of each NN
        for individual in population:
            individual.fitness = evaluate_fitness(x, y, individual)

        # keep the elites in the population
        elites = keep_elites(percentage_elites = 0.1, population = population)

        # select the NNs to keep
        selected = selection(population)

        # join the elites and selected NNs
        new_population = elites + selected

        # reproduce the population
        new_population = reproduce_population(population=new_population)

        # trim the population
        new_population = reduce_population(population=new_population, remove_percentage=0.17)

        # assign new population to the old population 
        population = new_population

    return population

In [19]:
NUM_GENERATIONS = 100
population = init_population(num_individuals=100)

if __name__ == "__main__":
    population = run_evolution(population=population, num_generations=NUM_GENERATIONS)

In [20]:
for individual in population:
    individual.fitness = evaluate_fitness(x, y, individual)

In [21]:
best_nn = population[0]
print(best_nn)

[[ 6.47896424  9.63355019]
 [ 9.69587566 -7.1825288 ]
 [-7.27831874  5.72926788]][[ 7.05052464  8.55239276 -2.22760167]
 [-6.31290666  6.38075773  7.42199817]][[1.11385709 9.15266254]][[ 5.66176621]
 [-9.83792108]
 [-6.87958631]][[6.70010708]
 [4.39449772]][[-3.83188627]]


In [22]:
def test_performance(x:Tuple[Tuple[int]], y: Tuple[int], nn:NN) -> None:
    
    for (x1, x2), y1 in zip(x, y):
        input_layer = np.array([[x1],[x2]])
        Z1 = np.add(np.matmul(nn.W_layer1, input_layer), nn.b_layer1)
        A1 = np.array([sigmoid(i) for i in Z1])
        A1 = normalize_vector(A1)
        Z2 = np.add(np.matmul(nn.W_layer2, A1), nn.b_layer2)
        A2 = np.array([sigmoid(i) for i in Z2])
        A2 = normalize_vector(A2)
        Z3 = np.add(np.matmul(nn.W_layer3, A2), nn.b_layer3)
        A3 = sigmoid(Z3)
        
        if A3 > 0.5:
            y_pred = 1
        else:
            y_pred = 0

        print(f"The inputs are {x1} and {x2}")
        print(f"The neural network predicted {y_pred}")
        print(f"The ground truth is {y1}\n")

In [23]:
test_performance(x = x, y = y, nn = best_nn)

The inputs are 0 and 0
The neural network predicted 0
The ground truth is 0

The inputs are 1 and 0
The neural network predicted 1
The ground truth is 1

The inputs are 0 and 1
The neural network predicted 1
The ground truth is 1

The inputs are 1 and 1
The neural network predicted 0
The ground truth is 0

